_한국어로 기계번역됨_


# 도구에서 그래프 상태를 업데이트하는 방법


!!! 정보 "전제 조건"
    이 가이드는 다음에 대한 이해를 전제로 합니다:
    
    - [Command](../../concepts/low_level/#command)

일반적인 사용 사례는 도구 내에서 그래프 상태를 업데이트하는 것입니다. 예를 들어, 고객 지원 애플리케이션에서는 대화 시작 시 고객 계좌 번호 또는 ID를 조회하고 싶을 수 있습니다. 도구에서 그래프 상태를 업데이트하려면 도구에서 `Command(update={"my_custom_key": "foo", "messages": [...]})`를 반환할 수 있습니다:

```python
@tool
def lookup_user_info(tool_call_id: Annotated[str, InjectedToolCallId], config: RunnableConfig):
    """사용자가 질문에 더 잘 답변할 수 있도록 사용자 정보를 조회하는 데 사용합니다."""
    user_info = get_user_info(config)
    return Command(
        update={
            # 상태 키 업데이트
            "user_info": user_info,
            # 메시지 기록 업데이트
            "messages": [ToolMessage("사용자 정보를 성공적으로 조회했습니다", tool_call_id=tool_call_id)]
        }
    )
```

!!! 중요

    `Command`를 반환하고 그래프 상태를 업데이트하는 도구를 사용하려면 미리 구축된 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent] / [`ToolNode`][langgraph.prebuilt.tool_node.ToolNode] 구성 요소를 사용하거나, 도구가 반환한 `Command` 객체를 수집하고 이를 목록으로 반환하는 사용자 정의 도구 실행 노드를 구현할 수 있습니다, 예를 들어:
    
    ```python
    def call_tools(state):
        ...
        commands = [tools_by_name[tool_call["name"]].invoke(tool_call) for tool_call in tool_calls]
        return commands
    ```

이 가이드는 LangGraph의 미리 구축된 구성 요소([`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent] / [`ToolNode`][langgraph.prebuilt.tool_node.ToolNode])를 사용하여 이를 수행하는 방법을 보여줍니다.

!!! 참고

    [`Command`][langgraph.types.Command]를 반환하는 도구에 대한 지원은 LangGraph `v0.2.59`에 추가되었습니다.

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정해 보겠습니다:


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import os
import getpass


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")


Please provide your OPENAI_API_KEY ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 식별하고 성능을 개선하세요. LangSmith는 트레이스 데이터를 사용하여 LangGraph로 구축된 LLM 앱을 디버깅하고 테스트하며 모니터링할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요.
    </p>
</div>


사용자 정보를 조회하고 사용자 정보에 따라 응답을 개인화할 수 있는 간단한 ReAct 스타일 에이전트를 만들어 봅시다.


## 도구 정의


먼저, 사용자 정보를 조회하는 데 사용할 도구를 정의해보겠습니다. 우리는 사전을 사용하여 사용자 정보를 단순히 조회하는 기초적인 구현을 사용할 것입니다:


In [ ]:
USER_INFO = [
    {"user_id": "1", "name": "Bob Dylan", "location": "New York, NY"},
    {"user_id": "2", "name": "Taylor Swift", "location": "Beverly Hills, CA"},
]

USER_ID_TO_USER_INFO = {info["user_id"]: info for info in USER_INFO}


In [3]:
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.types import Command
from langchain_core.tools import tool
from langchain_core.tools.base import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableConfig

from typing_extensions import Any, Annotated


class State(AgentState):
    # updated by the tool
    user_info: dict[str, Any]


@tool
def lookup_user_info(
    tool_call_id: Annotated[str, InjectedToolCallId], config: RunnableConfig
):
    """Use this to look up user information to better assist them with their questions."""
    user_id = config.get("configurable", {}).get("user_id")
    if user_id is None:
        raise ValueError("Please provide user ID")

    if user_id not in USER_ID_TO_USER_INFO:
        raise ValueError(f"User '{user_id}' not found")

    user_info = USER_ID_TO_USER_INFO[user_id]
    return Command(
        update={
            # update the state keys
            "user_info": user_info,
            # update the message history
            "messages": [
                ToolMessage(
                    "Successfully looked up user information", tool_call_id=tool_call_id
                )
            ],
        }
    )


## 프롬프트 정의


이제 개인화를 추가해 보겠습니다: 도구에서 상태가 업데이트된 후 상태 값에 따라 사용자에게 다르게 응답할 것입니다. 이를 달성하기 위해 그래프 상태에 따라 시스템 프롬프트를 동적으로 구성하는 함수를 정의합시다. 이 함수는 LLM이 호출될 때마다 호출되며, 함수의 출력은 LLM에 전달됩니다.


In [4]:
def prompt(state: State):
    user_info = state.get("user_info")
    if user_info is None:
        return state["messages"]

    system_msg = (
        f"User name is {user_info['name']}. User lives in {user_info['location']}"
    )
    return [{"role": "system", "content": system_msg}] + state["messages"]


## 그래프 정의


마지막으로, 이 모든 것을 사전 구축된 `create_react_agent`를 사용하여 단일 그래프로 결합해 보겠습니다:


In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

agent = create_react_agent(
    model,
    # pass the tool that can update state
    [lookup_user_info],
    state_schema=State,
    # pass dynamic prompt function
    prompt=prompt,
)


## 사용해보세요!


이제 우리의 에이전트를 실행해 보겠습니다. 도구가 어떤 정보를 찾을지 알 수 있도록 구성 파일에 사용자 ID를 제공해야 합니다.


In [6]:
for chunk in agent.stream(
    {"messages": [("user", "hi, what should i do this weekend?")]},
    # provide user ID in the config
    {"configurable": {"user_id": "1"}},
):
    print(chunk)
    print("\n")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7LSUh6ZDvGJAUvlWvXiCK4Gf', 'function': {'arguments': '{}', 'name': 'lookup_user_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 56, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9d50cd990b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-57eeb216-e35d-4501-aaac-b5c6b26fb17c-0', tool_calls=[{'name': 'lookup_user_info', 'args': {}, 'id': 'call_7LSUh6ZDvGJAUvlWvXiCK4Gf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 11, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

모델이 밥 딜런에게 뉴욕 활동을 올바르게 추천한 것을 볼 수 있습니다! 이제 테일러 스위프트에 대한 추천을 받아보겠습니다:


In [7]:
for chunk in agent.stream(
    {"messages": [("user", "hi, what should i do this weekend?")]},
    {"configurable": {"user_id": "2"}},
):
    print(chunk)
    print("\n")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5HLtJtzcgmKbtmK6By21wW5Y', 'function': {'arguments': '{}', 'name': 'lookup_user_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 56, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_c7ca0ebaca', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bacacd7d-76cc-4f6b-9e9b-d9e6f00b9391-0', tool_calls=[{'name': 'lookup_user_info', 'args': {}, 'id': 'call_5HLtJtzcgmKbtmK6By21wW5Y', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 11, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}